In [62]:
# from utils import load_dataset, load_grid_search_model
import pickle
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import spearmanr, pearsonr

In [14]:
PDBbind_minimized_features = [str(p) for p in list(Path('/pubhome/xli02/project/PLIM/deep_learning/RFscore/featured_data/whole_set/PDBbind_minimized').glob('*csv'))]
PDBbind_minimized_features.sort()
PDBbind_minimized_features

['/pubhome/xli02/project/PLIM/deep_learning/RFscore/featured_data/whole_set/PDBbind_minimized/PDBbind_minimized_xaa_RFscore_feature_6A.csv',
 '/pubhome/xli02/project/PLIM/deep_learning/RFscore/featured_data/whole_set/PDBbind_minimized/PDBbind_minimized_xab_RFscore_feature_6A.csv']

In [16]:
PLANet_all_features = [str(p) for p in list(Path('/pubhome/xli02/project/PLIM/deep_learning/RFscore/featured_data/whole_set/PLANet_all').glob('*csv'))]
PLANet_all_features.sort()
len(PLANet_all_features)

16

In [19]:
pdbbind_df = pd.concat([pd.read_csv(f, sep='\t') for f in PDBbind_minimized_features], ignore_index=True)
pdbbind_df

,6.6,6.7,6.8,6.16,7.6,7.7,7.8,7.16,8.6,8.7,...,53.16_2-4,53.16_4-6,vina_gauss1,vina_gauss2,vina_repulsion,vina_hydrophobic,vina_hydrogen,vina_num_rotors,pdb_id,-logAffi
0,151.0,52.0,40.0,0.0,18.0,7.0,6.0,0.0,94.0,36.0,...,0.0,0.0,72.490067,834.498657,4.866180,13.132306,3.644538,5.0,3zzf,0.40
1,268.0,59.0,55.0,0.0,9.0,3.0,2.0,0.0,13.0,3.0,...,0.0,0.0,64.783737,1151.906982,1.715198,57.094193,1.000000,6.0,3gww,0.45
2,123.0,35.0,13.0,6.0,15.0,7.0,2.0,1.0,15.0,7.0,...,0.0,0.0,36.497417,511.858124,1.093309,20.450666,1.012405,1.0,1w8l,0.49
3,150.0,39.0,47.0,0.0,13.0,4.0,7.0,0.0,55.0,19.0,...,0.0,0.0,64.599373,662.437805,4.373670,23.514214,1.229712,1.0,3fqa,0.49
4,62.0,13.0,9.0,0.0,65.0,18.0,15.0,0.0,74.0,18.0,...,0.0,0.0,56.117676,720.433594,4.767162,4.835436,4.067099,3.0,1zsb,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17174,376.0,111.0,119.0,0.0,17.0,11.0,6.0,0.0,99.0,56.0,...,0.0,0.0,130.402374,2028.301880,10.884005,58.562576,5.003764,14.0,7cpa,13.96
17175,704.0,93.0,150.0,0.0,119.0,12.0,19.0,0.0,0.0,0.0,...,0.0,0.0,194.077026,2491.811035,4.656355,145.388275,1.588944,12.0,2xuf,14.39
17176,286.0,34.0,49.0,0.0,53.0,9.0,10.0,0.0,79.0,18.0,...,0.0,0.0,86.293350,1188.106934,6.202013,26.014889,5.952804,5.0,1avd,15.00
17177,684.0,84.0,150.0,0.0,124.0,20.0,31.0,0.0,0.0,0.0,...,0.0,0.0,155.610001,2666.271973,3.567978,122.442429,1.428398,12.0,2xui,15.00


In [21]:
pdbbind_df.rename(columns={'pdb_id':'unique_identify'}, inplace=True)

In [20]:
planet_df = pd.concat([pd.read_csv(f, sep='\t') for f in PLANet_all_features], ignore_index=True)
planet_df

,6.6,6.7,6.8,6.16,7.6,7.7,7.8,7.16,8.6,8.7,...,53.16_2-4,53.16_4-6,vina_gauss1,vina_gauss2,vina_repulsion,vina_hydrophobic,vina_hydrogen,vina_num_rotors,unique_identify,-logAffi
0,260.0,36.0,44.0,0.0,53.0,7.0,11.0,0.0,69.0,15.0,...,0.0,0.0,87.591881,1165.407471,3.274903,33.328861,3.824730,5.0,CHEMBL1075026_1df8_CHEMBL857,14.000000
1,261.0,30.0,53.0,0.0,57.0,4.0,11.0,0.0,65.0,13.0,...,0.0,0.0,85.669991,1153.124878,3.930241,30.620073,4.318973,5.0,CHEMBL1075026_1n43_CHEMBL857,14.000000
2,242.0,28.0,47.0,0.0,45.0,7.0,10.0,0.0,64.0,14.0,...,0.0,0.0,79.032692,1087.753540,2.422066,28.773968,3.905863,5.0,CHEMBL1075026_1n9m_CHEMBL857,14.000000
3,267.0,35.0,48.0,0.0,54.0,8.0,10.0,0.0,70.0,15.0,...,0.0,0.0,89.217041,1155.619507,3.856589,31.429260,4.348562,5.0,CHEMBL1075026_1ndj_CHEMBL857,14.000000
4,249.0,29.0,49.0,0.0,51.0,8.0,13.0,0.0,63.0,12.0,...,0.0,0.0,78.034767,1138.325928,4.219721,26.825392,5.099978,5.0,CHEMBL1075026_1stp_CHEMBL857,14.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159056,390.0,49.0,55.0,13.0,110.0,19.0,16.0,4.0,30.0,5.0,...,0.0,0.0,123.841591,1597.889893,4.526520,74.793045,2.779781,6.0,CHEMBL6166_4zk5_CHEMBL3623139,8.795880
159057,380.0,40.0,69.0,12.0,51.0,8.0,8.0,3.0,48.0,9.0,...,0.0,0.0,97.371857,1389.387085,4.474770,77.596306,2.949464,3.0,CHEMBL6166_5di1_CHEMBL3754283,7.588909
159058,390.0,47.0,71.0,13.0,60.0,11.0,11.0,3.0,44.0,9.0,...,0.0,0.0,100.781845,1466.089233,3.770569,81.860435,1.998043,4.0,CHEMBL6166_5di1_CHEMBL3754304,6.285285
159059,90.0,19.0,13.0,0.0,21.0,4.0,3.0,0.0,109.0,24.0,...,0.0,0.0,43.404903,718.877625,5.071122,0.000000,3.605137,4.0,CHEMBL6175_2xml_CHEMBL90852,3.301030


In [22]:
features_df = pd.concat([pdbbind_df, planet_df], ignore_index=True)
features_df

,6.6,6.7,6.8,6.16,7.6,7.7,7.8,7.16,8.6,8.7,...,53.16_2-4,53.16_4-6,vina_gauss1,vina_gauss2,vina_repulsion,vina_hydrophobic,vina_hydrogen,vina_num_rotors,unique_identify,-logAffi
0,151.0,52.0,40.0,0.0,18.0,7.0,6.0,0.0,94.0,36.0,...,0.0,0.0,72.490067,834.498657,4.866180,13.132306,3.644538,5.0,3zzf,0.400000
1,268.0,59.0,55.0,0.0,9.0,3.0,2.0,0.0,13.0,3.0,...,0.0,0.0,64.783737,1151.906982,1.715198,57.094193,1.000000,6.0,3gww,0.450000
2,123.0,35.0,13.0,6.0,15.0,7.0,2.0,1.0,15.0,7.0,...,0.0,0.0,36.497417,511.858124,1.093309,20.450666,1.012405,1.0,1w8l,0.490000
3,150.0,39.0,47.0,0.0,13.0,4.0,7.0,0.0,55.0,19.0,...,0.0,0.0,64.599373,662.437805,4.373670,23.514214,1.229712,1.0,3fqa,0.490000
4,62.0,13.0,9.0,0.0,65.0,18.0,15.0,0.0,74.0,18.0,...,0.0,0.0,56.117676,720.433594,4.767162,4.835436,4.067099,3.0,1zsb,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176235,390.0,49.0,55.0,13.0,110.0,19.0,16.0,4.0,30.0,5.0,...,0.0,0.0,123.841591,1597.889893,4.526520,74.793045,2.779781,6.0,CHEMBL6166_4zk5_CHEMBL3623139,8.795880
176236,380.0,40.0,69.0,12.0,51.0,8.0,8.0,3.0,48.0,9.0,...,0.0,0.0,97.371857,1389.387085,4.474770,77.596306,2.949464,3.0,CHEMBL6166_5di1_CHEMBL3754283,7.588909
176237,390.0,47.0,71.0,13.0,60.0,11.0,11.0,3.0,44.0,9.0,...,0.0,0.0,100.781845,1466.089233,3.770569,81.860435,1.998043,4.0,CHEMBL6166_5di1_CHEMBL3754304,6.285285
176238,90.0,19.0,13.0,0.0,21.0,4.0,3.0,0.0,109.0,24.0,...,0.0,0.0,43.404903,718.877625,5.071122,0.000000,3.605137,4.0,CHEMBL6175_2xml_CHEMBL90852,3.301030


In [23]:
features_df.to_csv('/pubhome/xli02/project/PLIM/deep_learning/RFscore/featured_data/tow_datasets_features_6A.csv', sep='\t', index=False)

In [42]:
# feature_version = 'VR1'
model = 'RF'
feature_file = '/pubhome/xli02/project/PLIM/deep_learning/RFscore/featured_data/tow_datasets_features_6A.csv'

In [4]:
features = pd.read_csv(feature_file, sep='\t')
features

,6.6,6.7,6.8,6.16,7.6,7.7,7.8,7.16,8.6,8.7,...,53.16_2-4,53.16_4-6,vina_gauss1,vina_gauss2,vina_repulsion,vina_hydrophobic,vina_hydrogen,vina_num_rotors,unique_identify,-logAffi
0,151.0,52.0,40.0,0.0,18.0,7.0,6.0,0.0,94.0,36.0,...,0.0,0.0,72.490067,834.498657,4.866180,13.132306,3.644538,5.0,3zzf,0.400000
1,268.0,59.0,55.0,0.0,9.0,3.0,2.0,0.0,13.0,3.0,...,0.0,0.0,64.783737,1151.906982,1.715198,57.094193,1.000000,6.0,3gww,0.450000
2,123.0,35.0,13.0,6.0,15.0,7.0,2.0,1.0,15.0,7.0,...,0.0,0.0,36.497417,511.858124,1.093309,20.450666,1.012405,1.0,1w8l,0.490000
3,150.0,39.0,47.0,0.0,13.0,4.0,7.0,0.0,55.0,19.0,...,0.0,0.0,64.599373,662.437805,4.373670,23.514214,1.229712,1.0,3fqa,0.490000
4,62.0,13.0,9.0,0.0,65.0,18.0,15.0,0.0,74.0,18.0,...,0.0,0.0,56.117676,720.433594,4.767162,4.835436,4.067099,3.0,1zsb,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176235,390.0,49.0,55.0,13.0,110.0,19.0,16.0,4.0,30.0,5.0,...,0.0,0.0,123.841591,1597.889893,4.526520,74.793045,2.779781,6.0,CHEMBL6166_4zk5_CHEMBL3623139,8.795880
176236,380.0,40.0,69.0,12.0,51.0,8.0,8.0,3.0,48.0,9.0,...,0.0,0.0,97.371857,1389.387085,4.474770,77.596306,2.949464,3.0,CHEMBL6166_5di1_CHEMBL3754283,7.588909
176237,390.0,47.0,71.0,13.0,60.0,11.0,11.0,3.0,44.0,9.0,...,0.0,0.0,100.781845,1466.089233,3.770569,81.860435,1.998043,4.0,CHEMBL6166_5di1_CHEMBL3754304,6.285285
176238,90.0,19.0,13.0,0.0,21.0,4.0,3.0,0.0,109.0,24.0,...,0.0,0.0,43.404903,718.877625,5.071122,0.000000,3.605137,4.0,CHEMBL6175_2xml_CHEMBL90852,3.301030


In [5]:
vina_title =['vina_gauss1',
            'vina_gauss2',
            'vina_repulsion',
            'vina_hydrophobic',
            'vina_hydrogen',
            'vina_num_rotors']
rf_v1_title = features.columns.tolist()[:36]
# rf_v2_title = features.columns.tolist()[36:36+108]
feature_list = vina_title + rf_v1_title
# feature_list

# PDBbind_minimized_Rm_core_set

In [64]:
train = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_rm_core_ids/complex_6A/train.csv'
valid = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_rm_core_ids/complex_6A/valid.csv'
test = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_rm_core_ids/complex_6A/test.csv'

In [65]:
train_code = pd.read_csv(train, sep='\t', header=0, names=['unique_identify', '-logAffi'])
valid_code = pd.read_csv(valid, sep='\t', header=0, names=['unique_identify', '-logAffi'])
test_code = pd.read_csv(test, sep='\t', header=0, names=['unique_identify', '-logAffi'])
train_code

,unique_identify,-logAffi
0,3zzf,0.40
1,3gww,0.45
2,1w8l,0.49
3,3fqa,0.49
4,1zsb,0.60
...,...,...
13510,1amn,13.82
13511,7cpa,13.96
13512,2xuf,14.39
13513,2xui,15.00


In [66]:
train_set = pd.merge(train_code, features, on=['unique_identify','-logAffi'])
valid_set = pd.merge(valid_code, features, on=['unique_identify','-logAffi'])
test_set = pd.merge(test_code, features, on=['unique_identify','-logAffi'])

In [67]:
train_set

,unique_identify,-logAffi,6.6,6.7,6.8,6.16,7.6,7.7,7.8,7.16,...,53.8_4-6,53.16_0-2,53.16_2-4,53.16_4-6,vina_gauss1,vina_gauss2,vina_repulsion,vina_hydrophobic,vina_hydrogen,vina_num_rotors
0,3zzf,0.40,151.0,52.0,40.0,0.0,18.0,7.0,6.0,0.0,...,0.0,0.0,0.0,0.0,72.490067,834.498657,4.866180,13.132306,3.644538,5.0
1,3gww,0.45,268.0,59.0,55.0,0.0,9.0,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,64.783737,1151.906982,1.715198,57.094193,1.000000,6.0
2,1w8l,0.49,123.0,35.0,13.0,6.0,15.0,7.0,2.0,1.0,...,0.0,0.0,0.0,0.0,36.497417,511.858124,1.093309,20.450666,1.012405,1.0
3,3fqa,0.49,150.0,39.0,47.0,0.0,13.0,4.0,7.0,0.0,...,0.0,0.0,0.0,0.0,64.599373,662.437805,4.373670,23.514214,1.229712,1.0
4,1zsb,0.60,62.0,13.0,9.0,0.0,65.0,18.0,15.0,0.0,...,0.0,0.0,0.0,0.0,56.117676,720.433594,4.767162,4.835436,4.067099,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13510,1amn,13.82,220.0,46.0,52.0,0.0,20.0,3.0,5.0,0.0,...,0.0,0.0,0.0,0.0,75.580986,1142.393921,12.468986,33.350742,3.711069,3.0
13511,7cpa,13.96,376.0,111.0,119.0,0.0,17.0,11.0,6.0,0.0,...,0.0,0.0,0.0,0.0,130.402374,2028.301880,10.884005,58.562576,5.003764,14.0
13512,2xuf,14.39,704.0,93.0,150.0,0.0,119.0,12.0,19.0,0.0,...,0.0,0.0,0.0,0.0,194.077026,2491.811035,4.656355,145.388275,1.588944,12.0
13513,2xui,15.00,684.0,84.0,150.0,0.0,124.0,20.0,31.0,0.0,...,0.0,0.0,0.0,0.0,155.610001,2666.271973,3.567978,122.442429,1.428398,12.0


In [68]:
len(train_set) == len(train_code)

True

In [69]:
len(valid_set) == len(valid_code)

True

In [70]:
len(test_set) == len(test_code)

True

In [20]:
# train_num = train_set.shape[0]
# valid_num = valid_set.shape[0]
# train_num

13515

In [21]:
# valid_num

1689

In [71]:
train_valid_set =pd.concat([train_set, valid_set])
train_valid_set

,unique_identify,-logAffi,6.6,6.7,6.8,6.16,7.6,7.7,7.8,7.16,...,53.8_4-6,53.16_0-2,53.16_2-4,53.16_4-6,vina_gauss1,vina_gauss2,vina_repulsion,vina_hydrophobic,vina_hydrogen,vina_num_rotors
0,3zzf,0.40,151.0,52.0,40.0,0.0,18.0,7.0,6.0,0.0,...,0.0,0.0,0.0,0.0,72.490067,834.498657,4.866180,13.132306,3.644538,5.0
1,3gww,0.45,268.0,59.0,55.0,0.0,9.0,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,64.783737,1151.906982,1.715198,57.094193,1.000000,6.0
2,1w8l,0.49,123.0,35.0,13.0,6.0,15.0,7.0,2.0,1.0,...,0.0,0.0,0.0,0.0,36.497417,511.858124,1.093309,20.450666,1.012405,1.0
3,3fqa,0.49,150.0,39.0,47.0,0.0,13.0,4.0,7.0,0.0,...,0.0,0.0,0.0,0.0,64.599373,662.437805,4.373670,23.514214,1.229712,1.0
4,1zsb,0.60,62.0,13.0,9.0,0.0,65.0,18.0,15.0,0.0,...,0.0,0.0,0.0,0.0,56.117676,720.433594,4.767162,4.835436,4.067099,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,6cdj,11.70,524.0,123.0,160.0,0.0,57.0,13.0,24.0,0.0,...,0.0,0.0,0.0,0.0,159.876694,2503.994141,6.120054,62.741478,6.555303,13.0
1685,1mui,11.89,513.0,125.0,164.0,0.0,40.0,15.0,21.0,0.0,...,0.0,0.0,0.0,0.0,122.694466,2298.748535,2.728854,81.765160,3.566856,14.0
1686,3o9h,12.10,475.0,113.0,145.0,0.0,21.0,5.0,12.0,0.0,...,0.0,0.0,0.0,0.0,137.562210,2241.580566,4.930214,51.794060,4.841943,12.0
1687,2i0d,12.10,429.0,103.0,129.0,0.0,32.0,10.0,16.0,0.0,...,0.0,0.0,0.0,0.0,127.029976,2209.978027,4.000660,51.646614,4.006709,13.0


In [72]:
len(feature_list)

42

In [73]:
model = RandomForestRegressor(n_jobs=6,
                            verbose=0)

In [74]:
min_tree_feature = 3
max_tree_feature = min(9, len(feature_list)+1)

In [75]:
model_search = GridSearchCV(model, {
            "n_estimators":[100,200,300,400,500],
            "max_features":list(range(min_tree_feature, max_tree_feature))
        }, verbose=0, scoring="r2",cv=5)

In [76]:
model_search.fit(train_valid_set[feature_list], train_valid_set['-logAffi'])

GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=6),
             param_grid={'max_features': [3, 4, 5, 6, 7, 8],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='r2')

In [77]:
for param in model_search.best_params_.keys():
    print('PDBbind_minimized_Rm_core', 'VR1', param, model_search.best_params_[param])

PDBbind_minimized_Rm_core VR1 max_features 4
PDBbind_minimized_Rm_core VR1 n_estimators 300


In [79]:
sets = [
('Train', train_set['unique_identify'].tolist(), model_search.best_estimator_.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model_search.best_estimator_.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test', test_set['unique_identify'].tolist(), model_search.best_estimator_.predict(test_set[feature_list]), test_set['-logAffi'])]
pickle.dump(model_search.best_estimator_, open(f'/pubhome/xli02/project/PLIM/deep_learning/RFscore/searched_bead_parameter/PDBbind_minimized_Rm_core/RF_VR1_best_model.pkl', "wb"))

In [80]:
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    # plot_gaussian_kde_scatter_plot(pred, target, path=args["output_path"]+"/"+name+"_"+args["feature_version"]+"_"+args["model"]+".png")
    detail_result = pd.DataFrame({'unique_identify':pdb, 'pred_label':pred, 'true_label':target})
    detail_result.to_csv(f'/pubhome/xli02/project/PLIM/deep_learning/RFscore/searched_bead_parameter/PDBbind_minimized_Rm_core/detail_data_{name}_VR1_RF.csv', index=False)
results = pd.DataFrame(results, columns=['name','r2','Rp', 'mae'])
results.to_csv(f'/pubhome/xli02/project/PLIM/deep_learning/RFscore/searched_bead_parameter/PDBbind_minimized_Rm_core/final_result_VR1_RF.csv', index=False)

# PLANet_all_Rm_core

In [43]:
train = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/dataset/true_lig_alone/rm_core_ids/PLANet_all/complex_6A/train.csv'
valid = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/dataset/true_lig_alone/rm_core_ids/PLANet_all/complex_6A/valid.csv'
test = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/dataset/true_lig_alone/rm_core_ids/PLANet_all/complex_6A/test.csv'
train_code = pd.read_csv(train, sep='\t', header=0, names=['unique_identify', '-logAffi'])
valid_code = pd.read_csv(valid, sep='\t', header=0, names=['unique_identify', '-logAffi'])
test_code = pd.read_csv(test, sep='\t', header=0, names=['unique_identify', '-logAffi'])
train_code

,unique_identify,-logAffi
0,CHEMBL4630_2xez_CHEMBL1236804,0.180000
1,CHEMBL4630_2xf0_CHEMBL1236804,0.180000
2,CHEMBL235_3ads_CHEMBL1784945,0.207608
3,CHEMBL308_6gu4_CHEMBL80168,0.366532
4,CHEMBL205_1zfq_CHEMBL208108,0.602000
...,...,...
127151,CHEMBL1075026_1ndj_CHEMBL857,14.000000
127152,CHEMBL1075026_2izl_CHEMBL857,14.000000
127153,CHEMBL1075026_1stp_CHEMBL857,14.000000
127154,CHEMBL1075026_1n43_CHEMBL857,14.000000


In [44]:
train_set = pd.merge(train_code, features, on=['unique_identify','-logAffi'])
valid_set = pd.merge(valid_code, features, on=['unique_identify','-logAffi'])
test_set = pd.merge(test_code, features, on=['unique_identify','-logAffi'])
train_valid_set =pd.concat([train_set, valid_set])

In [45]:
len(train_set)

127138

In [46]:
len(train_valid_set)

143028

In [47]:
model = RandomForestRegressor(n_jobs=6,
                            verbose=0)
min_tree_feature = 3
max_tree_feature = min(9, len(feature_list)+1)

In [48]:
model_search = GridSearchCV(model, {
            "n_estimators":[100,200,300,400,500],
            "max_features":list(range(min_tree_feature, max_tree_feature))
        }, verbose=0, scoring="r2",cv=5)
model_search.fit(train_valid_set[feature_list], train_valid_set['-logAffi'])

GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=6),
             param_grid={'max_features': [3, 4, 5, 6, 7, 8],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='r2')

In [49]:
for param in model_search.best_params_.keys():
    print('PLANet_all_Rm_core', 'VR1', param, model_search.best_params_[param])

PLANet_all_Rm_core VR1 max_features 6
PLANet_all_Rm_core VR1 n_estimators 500


In [52]:
sets = [
('Train', train_set['unique_identify'].tolist(), model_search.best_estimator_.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model_search.best_estimator_.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test', test_set['unique_identify'].tolist(), model_search.best_estimator_.predict(test_set[feature_list]), test_set['-logAffi'])]

In [57]:
pickle.dump(model_search.best_estimator_, open(f'/pubhome/xli02/project/PLIM/deep_learning/RFscore/searched_bead_parameter/PLANet_all_Rm_core/RF_VR1_best_model.pkl', "wb"))

In [50]:
model_search.best_estimator_

RandomForestRegressor(max_features=6, n_estimators=500, n_jobs=6)

In [60]:
model_search.best_estimator_.predict(test_set[feature_list]).shape

(15894,)

In [63]:
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    # plot_gaussian_kde_scatter_plot(pred, target, path=args["output_path"]+"/"+name+"_"+args["feature_version"]+"_"+args["model"]+".png")
    detail_result = pd.DataFrame({'unique_identify':pdb, 'pred_label':pred, 'true_label':target})
    detail_result.to_csv(f'/pubhome/xli02/project/PLIM/deep_learning/RFscore/searched_bead_parameter/PLANet_all_Rm_core/detail_data_{name}_VR1_RF.csv', index=False)
results = pd.DataFrame(results, columns=['name','r2','Rp', 'mae'])
results.to_csv(f'/pubhome/xli02/project/PLIM/deep_learning/RFscore/searched_bead_parameter/PLANet_all_Rm_core/final_result_VR1_RF.csv', index=False)

# 2. Test_on_core

In [85]:
train_set

,unique_identify,-logAffi,6.6,6.7,6.8,6.16,7.6,7.7,7.8,7.16,...,53.8_4-6,53.16_0-2,53.16_2-4,53.16_4-6,vina_gauss1,vina_gauss2,vina_repulsion,vina_hydrophobic,vina_hydrogen,vina_num_rotors
0,3zzf,0.40,151.0,52.0,40.0,0.0,18.0,7.0,6.0,0.0,...,0.0,0.0,0.0,0.0,72.490067,834.498657,4.866180,13.132306,3.644538,5.0
1,3gww,0.45,268.0,59.0,55.0,0.0,9.0,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,64.783737,1151.906982,1.715198,57.094193,1.000000,6.0
2,1w8l,0.49,123.0,35.0,13.0,6.0,15.0,7.0,2.0,1.0,...,0.0,0.0,0.0,0.0,36.497417,511.858124,1.093309,20.450666,1.012405,1.0
3,3fqa,0.49,150.0,39.0,47.0,0.0,13.0,4.0,7.0,0.0,...,0.0,0.0,0.0,0.0,64.599373,662.437805,4.373670,23.514214,1.229712,1.0
4,1zsb,0.60,62.0,13.0,9.0,0.0,65.0,18.0,15.0,0.0,...,0.0,0.0,0.0,0.0,56.117676,720.433594,4.767162,4.835436,4.067099,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13510,1amn,13.82,220.0,46.0,52.0,0.0,20.0,3.0,5.0,0.0,...,0.0,0.0,0.0,0.0,75.580986,1142.393921,12.468986,33.350742,3.711069,3.0
13511,7cpa,13.96,376.0,111.0,119.0,0.0,17.0,11.0,6.0,0.0,...,0.0,0.0,0.0,0.0,130.402374,2028.301880,10.884005,58.562576,5.003764,14.0
13512,2xuf,14.39,704.0,93.0,150.0,0.0,119.0,12.0,19.0,0.0,...,0.0,0.0,0.0,0.0,194.077026,2491.811035,4.656355,145.388275,1.588944,12.0
13513,2xui,15.00,684.0,84.0,150.0,0.0,124.0,20.0,31.0,0.0,...,0.0,0.0,0.0,0.0,155.610001,2666.271973,3.567978,122.442429,1.428398,12.0


In [82]:
test_core = '/pubhome/xli02/Downloads/dataset/PDBbind/PDBbind_v2019/CASF_v16_index_dealt.csv'
core_df = pd.read_csv(test_core, sep='\t', header=0, names=['unique_identify', '-logAffi'])
core_df

,unique_identify,-logAffi
0,3ao4,2.07
1,3gv9,2.12
2,1uto,2.27
3,1ps3,2.28
4,4ddk,2.29
...,...,...
280,5dwr,11.22
281,4f2w,11.30
282,2x00,11.33
283,3o9i,11.82


In [84]:
core_set = pd.merge(core_df, features, on=['unique_identify','-logAffi'])
core_set

,unique_identify,-logAffi,6.6,6.7,6.8,6.16,7.6,7.7,7.8,7.16,...,53.8_4-6,53.16_0-2,53.16_2-4,53.16_4-6,vina_gauss1,vina_gauss2,vina_repulsion,vina_hydrophobic,vina_hydrogen,vina_num_rotors
0,3ao4,2.07,238.0,52.0,72.0,0.0,41.0,8.0,21.0,0.0,...,0.0,0.0,0.0,0.0,89.413338,1012.445923,3.515519,22.675829,2.582556,1.0
1,3gv9,2.12,100.0,32.0,31.0,0.0,16.0,5.0,4.0,0.0,...,0.0,0.0,0.0,0.0,42.500523,665.454651,3.699299,8.649817,1.566987,3.0
2,1uto,2.27,128.0,54.0,57.0,10.0,24.0,8.0,12.0,1.0,...,0.0,0.0,0.0,0.0,36.334339,568.007385,2.532124,4.822641,2.605420,2.0
3,1ps3,2.28,159.0,30.0,70.0,0.0,30.0,5.0,16.0,0.0,...,0.0,0.0,0.0,0.0,87.529808,1003.525940,9.235392,0.000000,11.397213,1.0
4,4ddk,2.29,185.0,77.0,40.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,62.257332,845.647339,0.912782,15.093544,0.509013,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,5dwr,11.22,420.0,36.0,59.0,0.0,66.0,6.0,13.0,0.0,...,0.0,0.0,0.0,0.0,92.461945,1721.970703,3.735585,72.053085,2.000000,5.0
281,4f2w,11.30,306.0,48.0,85.0,16.0,135.0,26.0,35.0,4.0,...,0.0,0.0,0.0,0.0,118.319153,1375.721313,3.114269,8.066344,4.417650,4.0
282,2x00,11.33,466.0,36.0,104.0,15.0,19.0,3.0,4.0,0.0,...,0.0,0.0,0.0,0.0,92.534737,1735.956543,1.925060,91.395836,0.000000,1.0
283,3o9i,11.82,473.0,108.0,146.0,0.0,38.0,8.0,19.0,0.0,...,0.0,0.0,0.0,0.0,144.244247,2239.065918,4.929898,53.110027,5.489359,12.0


In [89]:
train = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_rm_core_ids/complex_6A/train.csv'
valid = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_rm_core_ids/complex_6A/valid.csv'
test = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_rm_core_ids/complex_6A/test.csv'
train_code = pd.read_csv(train, sep='\t', header=0, names=['unique_identify', '-logAffi'])
valid_code = pd.read_csv(valid, sep='\t', header=0, names=['unique_identify', '-logAffi'])
test_code = pd.read_csv(test, sep='\t', header=0, names=['unique_identify', '-logAffi'])
train_set = pd.merge(train_code, features, on=['unique_identify','-logAffi'])
valid_set = pd.merge(valid_code, features, on=['unique_identify','-logAffi'])
test_set = pd.merge(test_code, features, on=['unique_identify','-logAffi'])

In [90]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=500,
        max_features=4,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])

RandomForestRegressor(max_features=4, min_samples_split=10, n_estimators=500,
                      n_jobs=6)

In [ ]:
sets = [
('Train', train_set['unique_identify'].tolist(), model_search.best_estimator_.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model_search.best_estimator_.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test', test_set['unique_identify'].tolist(), model_search.best_estimator_.predict(test_set[feature_list]), test_set['-logAffi'])]
pickle.dump(model_search.best_estimator_, open(f'/pubhome/xli02/project/PLIM/deep_learning/RFscore/searched_bead_parameter/PDBbind_minimized_Rm_core/RF_VR1_best_model.pkl', "wb"))

In [91]:
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]

In [92]:
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)
    # plot_gaussian_kde_scatter_plot(pred, target, path=args["output_path"]+"/"+name+"_"+args["feature_version"]+"_"+args["model"]+".png")
    # detail_result = pd.DataFrame({'unique_identify':pdb, 'pred_label':pred, 'true_label':target})
    # detail_result.to_csv(args["output_path"]+"/detail_data_"+name+"_"+args["feature_version"]+"_"+args["model"]+".csv", index=False)

['Train', '0.7947', '0.9200', '0.6546']
['Valid', '0.5194', '0.7311', '1.0056']
['Test_core', '0.5478', '0.7966', '1.1728']


# same para for PLANet_all

In [93]:
train = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/dataset/true_lig_alone/rm_core_ids/PLANet_all/complex_6A/train.csv'
valid = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/dataset/true_lig_alone/rm_core_ids/PLANet_all/complex_6A/valid.csv'
test = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/dataset/true_lig_alone/rm_core_ids/PLANet_all/complex_6A/test.csv'
train_code = pd.read_csv(train, sep='\t', header=0, names=['unique_identify', '-logAffi'])
valid_code = pd.read_csv(valid, sep='\t', header=0, names=['unique_identify', '-logAffi'])
test_code = pd.read_csv(test, sep='\t', header=0, names=['unique_identify', '-logAffi'])
train_set = pd.merge(train_code, features, on=['unique_identify','-logAffi'])
valid_set = pd.merge(valid_code, features, on=['unique_identify','-logAffi'])
test_set = pd.merge(test_code, features, on=['unique_identify','-logAffi'])

In [94]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=500,
        max_features=4,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])

RandomForestRegressor(max_features=4, min_samples_split=10, n_estimators=500,
                      n_jobs=6)

In [95]:
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]

In [96]:
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.8536', '0.9432', '0.3994']
['Valid', '0.6277', '0.8065', '0.6320']
['Test_core', '0.4955', '0.7627', '1.1983']


# change max_features

In [97]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=500,
        max_features=6,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.8686', '0.9490', '0.3762']
['Valid', '0.6368', '0.8105', '0.6220']
['Test_core', '0.4989', '0.7619', '1.1907']


In [98]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=800,
        max_features=6,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.8687', '0.9491', '0.3761']
['Valid', '0.6374', '0.8110', '0.6209']
['Test_core', '0.5027', '0.7644', '1.1864']


In [99]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=1000,
        max_features=7,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.8738', '0.9512', '0.3677']
['Valid', '0.6403', '0.8123', '0.6176']
['Test_core', '0.5069', '0.7673', '1.1790']


# change `max_feature`

In [105]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=1000,
        max_features=9,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.8815', '0.9541', '0.3552']
['Valid', '0.6441', '0.8137', '0.6131']
['Test_core', '0.5077', '0.7639', '1.1771']


# change `min_samples_split`

In [104]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=5, 
        n_estimators=1000,
        max_features=7,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.9248', '0.9729', '0.2786']
['Valid', '0.6555', '0.8201', '0.6001']
['Test_core', '0.5143', '0.7681', '1.1666']


In [106]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=2, 
        n_estimators=1000,
        max_features=7,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.9535', '0.9845', '0.2196']
['Valid', '0.6621', '0.8234', '0.5931']
['Test_core', '0.5190', '0.7707', '1.1605']


In [108]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=2, 
        n_estimators=500,
        max_features=7,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.9532', '0.9843', '0.2201']
['Valid', '0.6619', '0.8231', '0.5933']
['Test_core', '0.5178', '0.7688', '1.1592']


In [109]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=2, 
        n_estimators=1000,
        max_features=7,
        min_samples_leaf=20,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.6012', '0.8030', '0.6743']
['Valid', '0.5179', '0.7396', '0.7389']
['Test_core', '0.4403', '0.7346', '1.2803']


# 3. Test on core_inter_Uw

In [110]:
core_inter_Uw = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/test_on_core_set/2-core_intersected_Uw/core_intersected_Uw.csv'
core_inter_Uw_df = pd.read_csv(core_inter_Uw, sep='\t', header=0, names=['unique_identify', '-logAffi'])
core_inter_Uw_df

,unique_identify,-logAffi
0,1uto,2.27
1,3l7b,2.40
2,3udh,2.85
3,4llx,2.89
4,5aba,2.98
...,...,...
110,4gid,10.77
111,5c2h,11.09
112,1mq6,11.15
113,5dwr,11.22


In [111]:
core_inter_Uw_df_feature = pd.merge(core_inter_Uw_df, features, on=['unique_identify','-logAffi'])
core_inter_Uw_df_feature

,unique_identify,-logAffi,6.6,6.7,6.8,6.16,7.6,7.7,7.8,7.16,...,53.8_4-6,53.16_0-2,53.16_2-4,53.16_4-6,vina_gauss1,vina_gauss2,vina_repulsion,vina_hydrophobic,vina_hydrogen,vina_num_rotors
0,1uto,2.27,128.0,54.0,57.0,10.0,24.0,8.0,12.0,1.0,...,0.0,0.0,0.0,0.0,36.334339,568.007385,2.532124,4.822641,2.605420,2.0
1,3l7b,2.40,181.0,59.0,66.0,0.0,51.0,16.0,21.0,0.0,...,0.0,0.0,0.0,0.0,82.457764,1165.090210,3.701058,6.742232,6.496560,2.0
2,3udh,2.85,205.0,15.0,76.0,0.0,38.0,8.0,15.0,0.0,...,0.0,0.0,0.0,0.0,57.756817,780.527527,3.609698,39.569462,3.709292,0.0
3,4llx,2.89,112.0,7.0,15.0,2.0,64.0,5.0,9.0,0.0,...,0.0,0.0,0.0,0.0,41.777435,538.559387,1.087444,21.769232,1.932576,0.0
4,5aba,2.98,199.0,49.0,66.0,4.0,14.0,4.0,6.0,0.0,...,0.0,0.0,0.0,0.0,64.473930,909.060547,1.190412,18.465958,0.000000,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,4gid,10.77,656.0,154.0,215.0,0.0,82.0,24.0,32.0,0.0,...,0.0,0.0,0.0,0.0,217.109344,2639.540527,7.434659,92.202919,7.087039,17.0
111,5c2h,11.09,408.0,73.0,86.0,8.0,85.0,10.0,11.0,1.0,...,0.0,0.0,0.0,0.0,137.310455,1691.155884,2.291368,60.439640,1.000000,8.0
112,1mq6,11.15,396.0,102.0,107.0,16.0,86.0,20.0,26.0,3.0,...,0.0,0.0,0.0,0.0,155.300613,1695.887573,2.412405,22.720070,1.879444,10.0
113,5dwr,11.22,420.0,36.0,59.0,0.0,66.0,6.0,13.0,0.0,...,0.0,0.0,0.0,0.0,92.461945,1721.970703,3.735585,72.053085,2.000000,5.0


# PDBbind_minimized

In [123]:
train = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_rm_core_ids/complex_6A/train.csv'
valid = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_rm_core_ids/complex_6A/valid.csv'
test = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_rm_core_ids/complex_6A/test.csv'
train_code = pd.read_csv(train, sep='\t', header=0, names=['unique_identify', '-logAffi'])
valid_code = pd.read_csv(valid, sep='\t', header=0, names=['unique_identify', '-logAffi'])
test_code = pd.read_csv(test, sep='\t', header=0, names=['unique_identify', '-logAffi'])
train_set = pd.merge(train_code, features, on=['unique_identify','-logAffi'])
valid_set = pd.merge(valid_code, features, on=['unique_identify','-logAffi'])
test_set = pd.merge(test_code, features, on=['unique_identify','-logAffi'])

In [124]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=500,
        max_features=4,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.7944', '0.9198', '0.6547']
['Valid', '0.5204', '0.7318', '1.0040']
['Test_core', '0.5479', '0.7966', '1.1721']


In [125]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=500,
        max_features=4,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_inter_Uw_df_feature['unique_identify'].tolist(), model.predict(core_inter_Uw_df_feature[feature_list]), core_inter_Uw_df_feature['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.7945', '0.9199', '0.6543']
['Valid', '0.5182', '0.7303', '1.0053']
['Test_core', '0.5415', '0.8276', '1.1319']


In [126]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=500,
        max_features=8,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_inter_Uw_df_feature['unique_identify'].tolist(), model.predict(core_inter_Uw_df_feature[feature_list]), core_inter_Uw_df_feature['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.8235', '0.9318', '0.6021']
['Valid', '0.5298', '0.7355', '0.9935']
['Test_core', '0.5716', '0.8431', '1.0989']


# PDBbind_inter_Uw

In [119]:
train = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_intersected_Uw_rm_core_ids/complex_6A/train.csv'
valid = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_intersected_Uw_rm_core_ids/complex_6A/valid.csv'
test = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_intersected_Uw_rm_core_ids/complex_6A/test.csv'
train_code = pd.read_csv(train, sep='\t', header=0, names=['unique_identify', '-logAffi'])
valid_code = pd.read_csv(valid, sep='\t', header=0, names=['unique_identify', '-logAffi'])
test_code = pd.read_csv(test, sep='\t', header=0, names=['unique_identify', '-logAffi'])
train_set = pd.merge(train_code, features, on=['unique_identify','-logAffi'])
valid_set = pd.merge(valid_code, features, on=['unique_identify','-logAffi'])
test_set = pd.merge(test_code, features, on=['unique_identify','-logAffi'])

In [113]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=200,
        max_features=3,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.7700', '0.9129', '0.6178']
['Valid', '0.3777', '0.6185', '0.9525']
['Test_core', '0.4782', '0.7494', '1.2602']


In [114]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=200,
        max_features=3,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_inter_Uw_df_feature['unique_identify'].tolist(), model.predict(core_inter_Uw_df_feature[feature_list]), core_inter_Uw_df_feature['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.7703', '0.9131', '0.6179']
['Valid', '0.3768', '0.6175', '0.9507']
['Test_core', '0.5152', '0.7945', '1.1568']


In [120]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=500,
        max_features=8,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_inter_Uw_df_feature['unique_identify'].tolist(), model.predict(core_inter_Uw_df_feature[feature_list]), core_inter_Uw_df_feature['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.8166', '0.9309', '0.5479']
['Valid', '0.3991', '0.6340', '0.9308']
['Test_core', '0.5499', '0.8103', '1.1054']


In [122]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=2, 
        n_estimators=500,
        max_features=8,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_inter_Uw_df_feature['unique_identify'].tolist(), model.predict(core_inter_Uw_df_feature[feature_list]), core_inter_Uw_df_feature['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.9277', '0.9791', '0.3461']
['Valid', '0.4095', '0.6418', '0.9228']
['Test_core', '0.5542', '0.8076', '1.0993']


# PLANet_Uw

In [115]:
train = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/dataset/true_lig_alone/rm_core_ids/PLANet_Uw/complex_6A/train.csv'
valid = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/dataset/true_lig_alone/rm_core_ids/PLANet_Uw/complex_6A/valid.csv'
test = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/dataset/true_lig_alone/rm_core_ids/PLANet_Uw/complex_6A/test.csv'
train_code = pd.read_csv(train, sep='\t', header=0, names=['unique_identify', '-logAffi'])
valid_code = pd.read_csv(valid, sep='\t', header=0, names=['unique_identify', '-logAffi'])
test_code = pd.read_csv(test, sep='\t', header=0, names=['unique_identify', '-logAffi'])
train_set = pd.merge(train_code, features, on=['unique_identify','-logAffi'])
valid_set = pd.merge(valid_code, features, on=['unique_identify','-logAffi'])
test_set = pd.merge(test_code, features, on=['unique_identify','-logAffi'])

In [116]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=500,
        max_features=8,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.8426', '0.9430', '0.4197']
['Valid', '0.5599', '0.7626', '0.7109']
['Test_core', '0.4736', '0.7543', '1.2529']


In [117]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=500,
        max_features=8,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_inter_Uw_df_feature['unique_identify'].tolist(), model.predict(core_inter_Uw_df_feature[feature_list]), core_inter_Uw_df_feature['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.8424', '0.9428', '0.4200']
['Valid', '0.5611', '0.7635', '0.7096']
['Test_core', '0.5793', '0.8643', '1.0589']


In [118]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=200,
        max_features=3,
        verbose=0)
model.fit(train_set[feature_list], train_set["-logAffi"])
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Test_core', core_inter_Uw_df_feature['unique_identify'].tolist(), model.predict(core_inter_Uw_df_feature[feature_list]), core_inter_Uw_df_feature['-logAffi'])]
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    print(result)


['Train', '0.8008', '0.9271', '0.4775']
['Valid', '0.5415', '0.7549', '0.7304']
['Test_core', '0.5562', '0.8575', '1.1011']


# 4. scripts for predict

In [128]:
tvt_data_dir = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/PDBbind/pdbbind_v2019/minimized/dataset/PDBbind_minimized_rm_core_ids/complex_6A'
core_data = '/pubhome/xli02/Downloads/dataset/PDBbind/PDBbind_v2019/CASF_v16_index_dealt.csv'
core_intersected_Uw = '/pubhome/xli02/project/PLIM/deep_learning/FAST/fast_plim/test_on_core_set/2-core_intersected_Uw/core_intersected_Uw.csv'
feature_file = '/pubhome/xli02/project/PLIM/deep_learning/RFscore/featured_data/tow_datasets_features_6A.csv'
dataset_name = 'PDBbind_minimized_Rm_core'

In [139]:
rf_n_estimator = 500
rf_max_features = 8

In [130]:
features = pd.read_csv(feature_file, sep='\t')
vina_title =['vina_gauss1',
            'vina_gauss2',
            'vina_repulsion',
            'vina_hydrophobic',
            'vina_hydrogen',
            'vina_num_rotors']
rf_v1_title = features.columns.tolist()[:36]
# rf_v2_title = features.columns.tolist()[36:36+108]
feature_list = vina_title + rf_v1_title

In [131]:
len(feature_list)

42

In [133]:
feature_list[-5:]

['35.16', '53.6', '53.7', '53.8', '53.16']

In [135]:
train_code = pd.read_csv(f'{tvt_data_dir}/train.csv', sep='\t', header=0, names=['unique_identify', '-logAffi'])
valid_code = pd.read_csv(f'{tvt_data_dir}/valid.csv', sep='\t', header=0, names=['unique_identify', '-logAffi'])
self_test_code = pd.read_csv(f'{tvt_data_dir}/test.csv', sep='\t', header=0, names=['unique_identify', '-logAffi'])
core_code = pd.read_csv(core_data, sep='\t', header=0, names=['unique_identify', '-logAffi'])
core_inter_Uw_code = pd.read_csv(core_intersected_Uw, sep='\t', header=0, names=['unique_identify', '-logAffi'])

train_set = pd.merge(train_code, features, on=['unique_identify','-logAffi'])
valid_set = pd.merge(valid_code, features, on=['unique_identify','-logAffi'])
self_test_set = pd.merge(self_test_code, features, on=['unique_identify','-logAffi'])
core_set = pd.merge(core_code, features, on=['unique_identify','-logAffi'])
core_inter_Uw_set = pd.merge(core_inter_Uw_code, features, on=['unique_identify','-logAffi'])


In [138]:
core_set

,unique_identify,-logAffi,6.6,6.7,6.8,6.16,7.6,7.7,7.8,7.16,...,53.8_4-6,53.16_0-2,53.16_2-4,53.16_4-6,vina_gauss1,vina_gauss2,vina_repulsion,vina_hydrophobic,vina_hydrogen,vina_num_rotors
0,3ao4,2.07,238.0,52.0,72.0,0.0,41.0,8.0,21.0,0.0,...,0.0,0.0,0.0,0.0,89.413338,1012.445923,3.515519,22.675829,2.582556,1.0
1,3gv9,2.12,100.0,32.0,31.0,0.0,16.0,5.0,4.0,0.0,...,0.0,0.0,0.0,0.0,42.500523,665.454651,3.699299,8.649817,1.566987,3.0
2,1uto,2.27,128.0,54.0,57.0,10.0,24.0,8.0,12.0,1.0,...,0.0,0.0,0.0,0.0,36.334339,568.007385,2.532124,4.822641,2.605420,2.0
3,1ps3,2.28,159.0,30.0,70.0,0.0,30.0,5.0,16.0,0.0,...,0.0,0.0,0.0,0.0,87.529808,1003.525940,9.235392,0.000000,11.397213,1.0
4,4ddk,2.29,185.0,77.0,40.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,62.257332,845.647339,0.912782,15.093544,0.509013,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,5dwr,11.22,420.0,36.0,59.0,0.0,66.0,6.0,13.0,0.0,...,0.0,0.0,0.0,0.0,92.461945,1721.970703,3.735585,72.053085,2.000000,5.0
281,4f2w,11.30,306.0,48.0,85.0,16.0,135.0,26.0,35.0,4.0,...,0.0,0.0,0.0,0.0,118.319153,1375.721313,3.114269,8.066344,4.417650,4.0
282,2x00,11.33,466.0,36.0,104.0,15.0,19.0,3.0,4.0,0.0,...,0.0,0.0,0.0,0.0,92.534737,1735.956543,1.925060,91.395836,0.000000,1.0
283,3o9i,11.82,473.0,108.0,146.0,0.0,38.0,8.0,19.0,0.0,...,0.0,0.0,0.0,0.0,144.244247,2239.065918,4.929898,53.110027,5.489359,12.0


In [140]:
model = RandomForestRegressor(n_jobs=6,                            
        min_samples_split=10, 
        n_estimators=rf_n_estimator,
        max_features=rf_max_features,
        verbose=0)

model.fit(train_set[feature_list], train_set["-logAffi"])

RandomForestRegressor(max_features=8, min_samples_split=10, n_estimators=500,
                      n_jobs=6)

In [141]:
sets = [
('Train', train_set['unique_identify'].tolist(), model.predict(train_set[feature_list]), train_set['-logAffi']),
('Valid', valid_set['unique_identify'].tolist(), model.predict(valid_set[feature_list]), valid_set['-logAffi']),
('Self_test', self_test_set['unique_identify'].tolist(), model.predict(self_test_set[feature_list]), self_test_set['-logAffi']),
('Core', core_set['unique_identify'].tolist(), model.predict(core_set[feature_list]), core_set['-logAffi']),
('Core_intersected_Uw', core_inter_Uw_set['unique_identify'].tolist(), model.predict(core_inter_Uw_set[feature_list]), core_inter_Uw_set['-logAffi']),
]

In [151]:
from utils import plot_gaussian_kde_scatter_plot

In [152]:
output_path = '/pubhome/xli02/project/PLIM/deep_learning/RFscore/test_res'
feature_version = 'VR1'
model = 'RF'

In [153]:
results = []
for name, pdb, pred, target in sets:
    R2_score = r2_score(target, pred)
    Rp = pearsonr(target, pred)[0]
    mae = mean_absolute_error(target, pred)
    result = [name, format(R2_score,'.4f'), format(Rp,'.4f'), format(mae,'.4f')]
    results.append(result)
    res = f'R2:{R2_score:.4f}; Rp:{Rp:.4f}; mae:{mae:.4f}'
    plot_gaussian_kde_scatter_plot(dataset_name, name, res, pred, target, f'{output_path}/{dataset_name}/{name}_{feature_version}_{model}.png')
    detail_result = pd.DataFrame({'unique_identify':pdb, 'pred_label':pred, 'true_label':target})
    detail_result.to_csv(f'{output_path}/{dataset_name}/detail_data_{name}_{feature_version}_{model}.csv', index=False)
results = pd.DataFrame(results, columns=['name','r2','Rp', 'mae'])
results.to_csv(f'{output_path}/{dataset_name}/final_result_{feature_version}_{model}.csv', index=False)
pickle.dump(model, open(f'{output_path}/{dataset_name}/best_model_{feature_version}_{model}.pkl', "wb"))

TypeError: plot_gaussian_kde_scatter_plot() takes 3 positional arguments but 6 were given